In [1]:
from binary_radix_tree import *
from morton_code import *
from check import *
from morton_code import *
from util import *
import random

In [2]:
num_points = 100
num_bits_per_dim = 3
# Set random seed for reproducibility
random.seed(42)

# Generate random 3D points between 0 and 1
points = [(random.random(), random.random(), random.random()) for _ in range(num_points)]
# Filter to get only unique points
points = list(set(points))
print(f"num of unique points: {len(points)}")

# Convert points to Morton codes
morton_codes = []
for x, y, z in points:
    morton_code = coordinate_to_morton_code_3d(x, y, z, num_bits_per_dim)
    morton_codes.append(morton_code)

# Sort Morton codes
# Sort morton_codes and points together using morton_codes as the key
morton_codes, points = zip(*sorted(zip(morton_codes, points)))
morton_codes = list(morton_codes)
points = list(points)
print(f"morton codes: {morton_codes}")
offset = range(len(morton_codes))

# get unique points
same = [1] + [0 if morton_codes[i] == morton_codes[i-1] else 1 for i in range(1, len(morton_codes))]
offset = [offset[i] for i in range(len(morton_codes)) if same[i] == 1] + [len(morton_codes)]
print(f"offset: {offset}")
unique_mc = [morton_codes[i] for i in range(len(morton_codes)) if same[i] == 1]
print(f"unique_mc: {unique_mc}")

# create binary radix tree 
brt = BinaryRadixTree(unique_mc, num_bits_per_dim * 3)

# test 
query_point = [random.random(), random.random(), random.random()]
# radius = random.random()
radius = 0.5
print(f"ref_point: {query_point}, radius: {radius}")
check = check_distance(query_point, radius, num_bits_per_dim)
query_mc = brt.traverse_tree_distance(check)

# print queried mc
query_points = []
for mc, shifts in query_mc.items():
    for cell_shift in shifts:
        # print(f"mc idx: {mc}, cell_shift: {cell_shift}")
        for idx in range(offset[mc], offset[mc] + 1):
            shifted_point = (points[idx][0] + cell_shift[0], 
                           points[idx][1] + cell_shift[1],
                           points[idx][2] + cell_shift[2])
            if check.check_point(shifted_point):
                query_points.append(shifted_point)

# check result 
bench_points = []
for shift_z in range(check.cell_shift_z_min, check.cell_shift_z_max + 1):
    for shift_y in range(check.cell_shift_y_min, check.cell_shift_y_max + 1):
        for shift_x in range(check.cell_shift_x_min, check.cell_shift_x_max + 1):
            for point in points:
                shifted_point = (point[0] + shift_x, point[1] + shift_y, point[2] + shift_z)
                if check.check_point(shifted_point):
                    bench_points.append(shifted_point)

# Sort both lists of points and compare them
sorted_query_points = sorted(query_points)
sorted_bench_points = sorted(bench_points)

print("Query and benchmark points match:", sorted_query_points == sorted_bench_points)

if sorted_query_points != sorted_bench_points:
    print("Number of query points:", len(sorted_query_points))
    print("Number of benchmark points:", len(sorted_bench_points))
    # Find points that are in one list but not the other
    query_set = set(sorted_query_points)
    bench_set = set(sorted_bench_points)
    print("Points in query but not in benchmark:", query_set - bench_set)
    print("Points in benchmark but not in query:", bench_set - query_set)



num of unique points: 100
morton codes: [2, 3, 4, 4, 5, 5, 6, 7, 7, 8, 8, 9, 9, 9, 9, 10, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 14, 14, 17, 17, 18, 20, 22, 23, 23, 24, 24, 24, 24, 24, 24, 25, 25, 27, 27, 27, 28, 28, 29, 29, 31, 32, 32, 32, 32, 33, 33, 34, 36, 36, 36, 37, 37, 37, 38, 38, 38, 39, 40, 41, 41, 43, 43, 43, 45, 46, 46, 48, 48, 48, 49, 50, 51, 52, 53, 54, 54, 55, 55, 55, 57, 57, 57, 60, 60, 61, 61, 61, 62, 63]
offset: [0, 1, 2, 4, 6, 7, 9, 11, 15, 16, 19, 24, 26, 28, 30, 31, 32, 33, 35, 41, 43, 46, 48, 50, 51, 55, 57, 58, 61, 64, 67, 68, 69, 71, 74, 75, 77, 80, 81, 82, 83, 84, 85, 87, 90, 93, 95, 98, 99, 100]
unique_mc: [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 17, 18, 20, 22, 23, 24, 25, 27, 28, 29, 31, 32, 33, 34, 36, 37, 38, 39, 40, 41, 43, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 57, 60, 61, 62, 63]
ref_point: [0.9039285715598931, 0.5455902892055223, 0.8345950198860167], radius: 0.5
Query and benchmark points match: False
Number of query points: 30
Number of benchmark